In [119]:
#!pip install ucimlrepo

<h1><b> Importing Data</b> </h1>

In [120]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [121]:
from ucimlrepo import fetch_ucirepo 
import pandas as pd
# fetch dataset 
census_income = fetch_ucirepo(id=20) 


In [122]:
df = pd.DataFrame(census_income.data.original)

In [123]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


<b><h1> Data Cleaning</h1></b> 

**Removing whitespaces from object type columns**

In [124]:
#strip string columns

for col in census_income.data.original.columns:
    if df[col].dtype in ['O']:
        df[col]=df[col].str.strip()

In [125]:
#Check for Data frame info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             48842 non-null  int64 
 1   workclass       47879 non-null  object
 2   fnlwgt          48842 non-null  int64 
 3   education       48842 non-null  object
 4   education-num   48842 non-null  int64 
 5   marital-status  48842 non-null  object
 6   occupation      47876 non-null  object
 7   relationship    48842 non-null  object
 8   race            48842 non-null  object
 9   sex             48842 non-null  object
 10  capital-gain    48842 non-null  int64 
 11  capital-loss    48842 non-null  int64 
 12  hours-per-week  48842 non-null  int64 
 13  native-country  48568 non-null  object
 14  income          48842 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


<b><h1> Handle missing data </h1></b>

In [126]:
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', '?', 'Self-emp-inc', 'Without-pay', 'Never-worked',
       nan], dtype=object)

**`?` It expresses as missing data. so chaning `?` to `NaN`**

In [129]:
#changing '?' to NaN
df.replace("?", np.NaN, inplace=True)

In [131]:
#checking for missing values
data_na = df.isna().sum()
data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)


occupation        2809
workclass         2799
native-country     857
dtype: int64

In [137]:
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', nan, 'Self-emp-inc', 'Without-pay', 'Never-worked'],
      dtype=object)

In [141]:
df['workclass'].value_counts()

Private             33906
Self-emp-not-inc     3862
Local-gov            3136
State-gov            1981
Self-emp-inc         1695
Federal-gov          1432
Without-pay            21
Never-worked           10
Name: workclass, dtype: int64

**There are 2799 rows in the `Occupation` column and the `Workclass` that are missing. But `Occupation` column has 10 missing data more becouse the person doesn't work.**

**So we replace these data with `No-occupation`**

In [143]:
df[df['workclass']=='Never-worked']= df[df['workclass']=='Never-worked'].fillna('No-occupation')


In [144]:
data_na = df.isna().sum()
data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)

workclass         2799
occupation        2799
native-country     857
dtype: int64

**The `workclas` column and the `occupation` column both contain `2799` Null values**

In [146]:
df['native-country'].value_counts()[:5]

United-States    43832
Mexico             951
Philippines        295
Germany            206
Puerto-Rico        184
Name: native-country, dtype: int64

**As you can see `90%` of the data in the `"native country"` column consists of `"United States"`, it may be appropriate to fill the null values in that column with `"United States"` to maintain data consistency.**

In [147]:
df['native-country'].fillna('United-States', inplace=True)

In [148]:
# checking the missing values
data_na = df.isna().sum()
data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)

workclass     2799
occupation    2799
dtype: int64

**Droping the Null values**

In [149]:
df.dropna(inplace=True)

**`Capital Gain & loss Columns`**

In [150]:
len1, len2 = len(df[df['capital-gain']== 0])/len(df), len(df[df['capital-loss']== 0])/len(df)

print(round(len1,2),'%', round(len2,2),'%')

0.92 % 0.95 %


**Most of `90%` of Capital Gain and loss columns equal `0`.These columns may be difficult to model. Machine learning models often have difficulty learning from these columns and make the model less accurate.**

In [151]:
# Droping the Capital-gain and Capital-loss columns
df.drop(['capital-gain', 'capital-loss'], axis=1, inplace=True)

In [152]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K


**Checking the income column**

In [153]:
df['income'].unique()

array(['<=50K', '>50K', '<=50K.', '>50K.'], dtype=object)

**Income column has some value with `.` let's correct that**

In [154]:
df.replace({'income':{">50K.":">50K" , "<=50K.":"<=50K"}},inplace=True)

In [155]:
(df['native-country'].value_counts()/len(df))[:5]

United-States    0.914645
Mexico           0.019612
Philippines      0.006146
Germany          0.004192
Puerto-Rico      0.003801
Name: native-country, dtype: float64

**Most of `90%` of the `Native Country` is  `United-States`. That make our model less accurate and misleading.**

**let's drop this column**

In [156]:
df.drop(columns='native-country', inplace=True)